In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from collections import OrderedDict
import numpy as np
import timm
from codecarbon import EmissionsTracker


tracker = EmissionsTracker()
tracker.start()


def get_vit_model(num_classes):
    model = timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=num_classes)
    return model

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss(reduction='sum')(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return test_loss / len(test_loader.dataset), correct / len(test_loader.dataset)

def average_weights(weight_list):
    avg_weights = OrderedDict()
    for key in weight_list[0].keys():
        avg_weights[key] = sum([client_weights[key] for client_weights in weight_list]) / len(weight_list)
    return avg_weights
    
def create_client_dataset(dataset, num_clients):
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)
    client_indices = np.array_split(indices, num_clients)
    client_datasets = [Subset(dataset, index_list) for index_list in client_indices]
    return client_datasets



num_clients = 5
client_datasets = create_client_dataset(train_dataset, num_clients)
train_loaders = [DataLoader(client_dataset, batch_size=100, shuffle=True) for client_dataset in client_datasets]
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10









def federated_learning(train_loaders, test_loader, device, epochs, num_clients, communication_rounds, client_weighting, clients_per_round):
    global_model = get_vit_model(10).to(device)
    client_models = [get_vit_model(10).to(device) for _ in range(num_clients)]

    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    for com_round in range(1, communication_rounds + 1):
        print(f'Communication round {com_round}/{communication_rounds}')


        # select new randomised clients 
        selected_clients = np.random.choice(range(num_clients), size=clients_per_round, replace=False)
        client_weights = []

        for idx in selected_clients:
            client_model = client_models[idx]
            optimizer = optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9)
            train_model(client_model, device, train_loaders[idx], optimizer, epochs)

            client_weight = client_weighting[idx] if client_weighting else 1
            weighted_client_state = OrderedDict()

            for key in client_model.state_dict().keys():
                weighted_client_state[key] = client_model.state_dict()[key] * client_weight

            client_weights.append(weighted_client_state)

        global_weights = average_weights(client_weights)
        global_model.load_state_dict(global_weights)

        test_loss, accuracy = test_model(global_model, device, test_loader)
        print(f'Test loss: {test_loss:.4f}, Accuracy: {accuracy * 100:.2f}%\n')

    return global_model



communication_rounds = 50
client_weighting = [1, 1, 1, 1, 1]  # Equal weighting for all clients
clients_per_round = 5

federated_model = federated_learning(train_loaders, test_loader, device, epochs, num_clients, communication_rounds, client_weighting, clients_per_round)


tracker.stop()


[codecarbon INFO @ 00:45:14] [setup] RAM Tracking...
[codecarbon INFO @ 00:45:14] [setup] GPU Tracking...
[codecarbon INFO @ 00:45:14] No GPU found.
[codecarbon INFO @ 00:45:14] [setup] CPU Tracking...
[codecarbon ERROR @ 00:45:14] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarbon/issues/244 for workarounds : [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl/intel-rapl:1/energy_uj'
[codecarbon ERROR @ 00:45:14] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarbon/issues/244 for workarounds : [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl/intel-rapl:0/energy_uj'
[codecarbon INFO @ 00:45:14] Tracking Intel CPU via RAPL interface
[codecarbon ERROR @ 00:45:15] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarb

Files already downloaded and verified
Files already downloaded and verified
Communication round 1/50


[codecarbon INFO @ 00:45:33] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 00:45:33] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 00:45:33] 0.000024 kWh of electricity used since the begining.
[codecarbon INFO @ 00:45:48] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 00:45:48] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 00:45:48] 0.000048 kWh of electricity used since the begining.
[codecarbon INFO @ 00:46:03] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 00:46:03] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 00:46:03] 0.000072 kWh of electricity used since the begining.
[codecarbon INFO @ 00:46:18] Energy consumed for RAM : 0.000096 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 00:46:18] Energy consumed for all CPUs :

Test loss: 0.1197, Accuracy: 96.03%

Communication round 2/50


[codecarbon INFO @ 01:19:03] Energy consumed for RAM : 0.003237 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 01:19:03] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 01:19:03] 0.003237 kWh of electricity used since the begining.
[codecarbon INFO @ 01:19:18] Energy consumed for RAM : 0.003261 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 01:19:18] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 01:19:18] 0.003261 kWh of electricity used since the begining.
[codecarbon INFO @ 01:19:33] Energy consumed for RAM : 0.003285 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 01:19:33] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 01:19:33] 0.003285 kWh of electricity used since the begining.
[codecarbon INFO @ 01:19:48] Energy consumed for RAM : 0.003309 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 01:19:48] Energy consumed for all CPUs :

...

[codecarbon INFO @ 09:37:19] Energy consumed for RAM : 0.051022 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 09:37:19] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 09:37:19] 0.051022 kWh of electricity used since the begining.
[codecarbon INFO @ 09:37:34] Energy consumed for RAM : 0.051046 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 09:37:34] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 09:37:34] 0.051046 kWh of electricity used since the begining.
[codecarbon INFO @ 09:37:49] Energy consumed for RAM : 0.051070 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 09:37:49] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 09:37:49] 0.051070 kWh of electricity used since the begining.
[codecarbon INFO @ 09:38:04] Energy consumed for RAM : 0.051094 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 09:38:04] Energy consumed for all CPUs :

Test loss: 0.1288, Accuracy: 96.57%

Communication round 19/50


[codecarbon INFO @ 10:08:49] Energy consumed for RAM : 0.054043 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:08:49] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 10:08:49] 0.054043 kWh of electricity used since the begining.
[codecarbon INFO @ 10:09:04] Energy consumed for RAM : 0.054067 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:09:04] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 10:09:04] 0.054067 kWh of electricity used since the begining.
[codecarbon INFO @ 10:09:19] Energy consumed for RAM : 0.054091 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:09:19] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 10:09:19] 0.054091 kWh of electricity used since the begining.
[codecarbon INFO @ 10:09:34] Energy consumed for RAM : 0.054115 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:09:34] Energy consumed for all CPUs :

Test loss: 0.1296, Accuracy: 96.57%

Communication round 20/50


[codecarbon INFO @ 10:42:04] Energy consumed for RAM : 0.057232 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:42:04] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 10:42:04] 0.057232 kWh of electricity used since the begining.
[codecarbon INFO @ 10:42:19] Energy consumed for RAM : 0.057256 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:42:19] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 10:42:19] 0.057256 kWh of electricity used since the begining.
[codecarbon INFO @ 10:42:34] Energy consumed for RAM : 0.057280 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:42:34] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 10:42:34] 0.057280 kWh of electricity used since the begining.
[codecarbon INFO @ 10:42:49] Energy consumed for RAM : 0.057304 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 10:42:49] Energy consumed for all CPUs :